In [29]:
import re
from bs4 import BeautifulSoup
from collections import defaultdict


In [46]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait as ww
from bs4 import BeautifulSoup as bs
from collections import defaultdict
import time

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

base_url = "https://www.yna.co.kr/news/{}"
article_base = "https://www.yna.co.kr/"

dict = defaultdict(list)
for page in range(1, 2):
    url = base_url.format(page)
    driver.get(url)
    time.sleep(2)

    # 기사 제목과 링크 추출
    
    cid_list = []
    li_list = driver.find_elements(By.CSS_SELECTOR, 'li[data-cid]')

    for li in li_list:
        cid_value = li.get_attribute('data-cid')
        if cid_value and cid_value.startswith('AKR20251126'):
            web_id = cid_value.replace('AKR20251126', '').strip()
            article_url = 'https://www.yna.co.kr/view/AKR20251126' + web_id
            
        # open new window with article page
        driver.execute_script('window.open(arguments[0]);', article_url)
        driver.switch_to.window(driver.window_handles[1])

        article_soup = bs(driver.page_source, 'html.parser')

        genre_tag = article_soup.select_one('meta[itemprop="genre"]')

        if genre_tag and genre_tag.has_attr('content'):
            genre_value = genre_tag['content'].strip()
            dict['Category'].append(genre_value)
        else:
            dict['Category'].append(None)

        
        dict['Title'].append(article_soup.select_one('#container > div.container591 > div.content90 > header > h1').get_text(strip=True))
        #dict['Date'].append(article_soup.select_one('#articleViewCon > article > header > ul > li:nth-child(2)').get_text(strip=True))
        date_tag = article_soup.select_one('p.ir-txt01')
        if date_tag:
            date_text = date_tag.get_text(strip=True)

    # 정규표현식으로 연도, 월, 일 추출
            match = re.search(r'(\d{4})년(\d{1,2})월(\d{1,2})일', date_text)
            if match:
                year, month, day = match.groups()
                formatted_date = f"{year}-{month.zfill(2)}-{day.zfill(2)}"
                dict['Date'].append(formatted_date)
            else:
                dict['Date'].append(None)
        else:
            dict['Date'].append(None)



    

        sentences = []

        atc = article_soup.select_one('#articleWrap > div.story-news.article')
        p_tags = atc.find_all('p')[:-2]

        for p_tag in p_tags:
            sentences.append(p_tag.get_text(strip = True))



        dict['Contents'].append(' '.join(sentences))

        driver.close()
        driver.switch_to.window(driver.window_handles[0])
driver.quit()

print(dict)




defaultdict(<class 'list'>, {'Category': ['정치', '스포츠', '국제', '사회', '사회', '사회', '경제', '스포츠', '사건사고', '경제', '정치', '사회', '정치', '정치', '사회', '경제', '국제', '사회', '사회', '정치', '정치', '국제', '정치', '국제', '산업'], 'Title': ['국립대 병원·치과병원 소관 교육→복지부로…국회소위 통과(종합)', '귀화 선수 모아이즈, 레슬링 국가대표 최종 선발전 진출', '中, 소비 촉진 지원책…반려동물·장난감 등 포함', '충북대-교통대, 최종 통합협의안 합의…"주요 쟁점 사실상 정리"', '현대제철 당진제철소 불법파견 소송 2심서 566명만 정규직 인정', '"김건희 그라프 목걸이는 한학자 지침으로 준 尹 취임 선물"', 'JP모건, 넥스트레이드 본격 참여하나…코스콤, 관련 전산 개발', '도쿄 데플림픽 폐회…한국, 금메달 11개로 목표 초과 달성', '청주 실종여성 살해 혐의 전 연인 긴급체포…행방불명 43일만(종합)', '구윤철, UAE국경일 기념행사 참석…"공동번영 나아가자"', '\'항소포기국조 법사위\' 국힘 수용에 與 "논의"…세부조건엔 이견(종합2보)', '교원3단체 "평가원 고교학점제 설문 결과, 현장과 괴리 커"(종합)', '기아차 공장 찾은 金총리 "광주, 모빌리티 거점 될 것"', '유병호 "감사원 고발은 명백히 위법부당…TF 일방적 판단"', '경찰, \'신체 발언\' 이준석 무혐의 확정…"결정에 문제 없어"', "'잠재력 극대화' 아프리카투자포럼 26∼28일 모로코서 개막", '프랑스서 러 간첩 활동 의심 인도주의 단체 적발', '\'약물운전 혐의\' 벽산그룹 3세 1심 무죄…"사고원인 단정 못해"', '한덕수 "尹 계엄 결정 돌리려 했으나 역부족"…1월 21일 선고(종합)', '국민의힘, 신임 인권위원 후보로 김학자 변호사 추천', '한-태국 전략대화…온라인 스캠 등 초국가범죄 대응 긴밀 소통(종합)', '러

In [47]:
import json

articles = []
for i in range(len(dict['Category'])):
    article = {
        "num": i+1,
        "Category": dict['Category'][i],
        "Date": dict['Date'][i],
        "Title": dict['Title'][i],
        "Contents": dict['Contents'][i]
    }
    articles.append(article)

# JSON으로 저장
with open('articles_yna.json', 'w', encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)
